# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902003


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:02<00:41,  1.44s/it]

Rendering models:  13%|█▎        | 4/31 [00:04<00:33,  1.26s/it]

Rendering models:  16%|█▌        | 5/31 [00:07<00:46,  1.79s/it]

Rendering models:  19%|█▉        | 6/31 [00:07<00:32,  1.31s/it]

...

Rendering models:  90%|█████████ | 28/31 [00:11<00:00,  6.72it/s]

Rendering models:  94%|█████████▎| 29/31 [00:11<00:00,  5.26it/s]

Rendering models:  97%|█████████▋| 30/31 [00:12<00:00,  4.66it/s]

Rendering models: 100%|██████████| 31/31 [00:12<00:00,  5.37it/s]

equidad1                              0.000218
bkbolli                               0.000145
bobhail                               0.000119
not-logged-in-bfa0f2700a26d5100f69    0.000195
not-logged-in-a87793e85210b5433279    0.000312
bronzesword                           0.000165
LeonardoDB                            0.243942
not-logged-in-ef6618747dba5016be4d    0.196604
messier1                              0.002010
Jboy2000                              0.001988
defender1969                          0.000876
lore98                                0.000108
not-logged-in-9a45911eb71184fe7c7f    0.002422
not-logged-in-5f30b2bb9b7f9d49da41    0.028546
leclercsimon                          0.000116
clairedeu                             0.000123
not-logged-in-73559e43095c8a8cdb85    0.000360
not-logged-in-3ae935e3afaee843f530    0.001339
alexmnelson18                         0.000089
Lavadude                              0.070698
not-logged-in-e991c008411b56a8790d    3.285690
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())